In [2]:
from decimal import ROUND_HALF_UP, Decimal

import numpy as np
import pandas as pd

In [3]:
df_price = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv')
df_financial = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv')
df_stocklist = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv')

/var/folders/1k/wj0bvnz165sfyvwsqlfxwfmw6bk41g/T/ipykernel_98184/1116028146.py:2: DtypeWarning: Columns (14,15,16,17,18,19,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_financial = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv')


In [4]:
def adjust_price(price):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated AdjustedClose
    """
    # transform Date column into datetime
    price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")

    def generate_adjusted_close(df):
        
        # sort data to generate CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        # generate CumulativeAdjustmentFactor
        df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()
        # generate AdjustedClose
        df.loc[:, "AdjustedClose"] = (
            df["CumulativeAdjustmentFactor"] * df["Close"]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        # reverse order
        df = df.sort_values("Date")
        # to fill AdjustedClose, replace 0 into np.nan
        df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
        # forward fill AdjustedClose
        df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()
        return df

    # generate AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted_close).reset_index(drop=True)

    # price.set_index("Date", inplace=True)
    return price

In [5]:
price_cols = ['Close']

df_price[price_cols] = df_price[price_cols].replace(0.0, np.nan)

# add high/low
df_price["diff_high_low"] = (
    np.log1p(df_price["High"]) / np.log1p(df_price["Low"])
    ).astype("float16")
    
log_price = df_price[price_cols].apply(np.log1p)
diff_log_price = log_price.diff()
diff_log_price['SecuritiesCode'] = df_price.SecuritiesCode

# add new features
fea_price = [df_price]

group_code = df_price.groupby("SecuritiesCode")
price_gb_code = df_price.groupby("SecuritiesCode")[price_cols]
diff_log_price_gb_code = diff_log_price.groupby("SecuritiesCode")[price_cols]

for span in [20, 40, 60, 120]:
    pct_change = (
        price_gb_code.pct_change(span)
            .add_prefix(f"feat_pct{span}_")
            .astype('float16')
    )
    volatility = (
        diff_log_price_gb_code.rolling(span, min_periods=1)
            .std()
            .reset_index("SecuritiesCode", drop=True)
    )
    volatility = (
        (log_price / volatility).add_prefix(f"feat_vl{span}_")
            .astype('float16')
    )
    ma = (
        price_gb_code.rolling(span, min_periods=1)
            .mean()
            .reset_index("SecuritiesCode", drop=True)
    )
    ma = (df_price[price_cols] / ma).add_prefix(f"feat_rmr{span}_").astype('float16')
    diff_roll = (
        group_code.diff_high_low
            .rolling(span, min_periods=1)
            .mean().
            rename(f"feat_wd{span}")
            .reset_index("SecuritiesCode", drop=True)
            .astype('float16')
    )
    fea_price += [pct_change, volatility, ma, diff_roll]

df_price_fe = pd.concat(fea_price, axis=1)
df_price_fe = adjust_price(df_price_fe) # add adjusted close value

In [215]:
df_fn = df_financial.copy()
acc_cols = ['NetSales', 'Profit', 'OperatingProfit', 'OrdinaryProfit']
df_fn[acc_cols] = df_fn[acc_cols].replace('－', 0).astype('float32')
# drop irregular financial quarter
df_fn = df_fn[~df_fn.TypeOfCurrentPeriod.isin(['4Q', '5Q', np.nan])].reset_index(drop=True)
df_fn['quarter'] = df_fn.TypeOfCurrentPeriod.map({"1Q": 1, "2Q": 2, "3Q": 3, "FY":4})
df_fn['FiscalYear'] = df_fn.CurrentFiscalYearEndDate.str.split('-', expand=True)[0].astype('int16')

fp_ymd = df_fn.CurrentPeriodEndDate.str.split('-', expand=True).astype('float32')
df_fn['fp_month'] = (fp_ymd[0] - 2016) * 12 + fp_ymd[1]
df_quarter = (
    df_fn[~(df_fn.CurrentPeriodEndDate.isna())]
        .groupby(['SecuritiesCode', 'CurrentPeriodEndDate'])
        .nth(-1)
)

df_quarter['quarter_span'] = df_quarter.groupby('SecuritiesCode').fp_month.diff().fillna(3)
df_fn = df_fn.join(df_quarter.quarter_span, on=['SecuritiesCode', 'CurrentPeriodEndDate'])
year_span = (df_quarter.groupby(['SecuritiesCode', 'FiscalYear']).quarter_span.sum()).rename('year_span')
df_fn = df_fn.join(year_span, on=['SecuritiesCode', 'FiscalYear'])
df_diff = df_quarter.groupby(['SecuritiesCode', 'FiscalYear'])[acc_cols].diff().fillna(
    df_quarter[acc_cols] / df_quarter.quarter.values[:, None]
)
df_diff /= (df_quarter["quarter_span"].values[:, None] / 3)

In [216]:
df_ma = df_diff[acc_cols].rolling(4, min_periods=1).mean()
df_diff = df_diff.add_prefix('diff_')
df_ma = df_ma.add_prefix('ma_')

df_fn = df_fn.merge(df_diff, on=['SecuritiesCode', 'CurrentPeriodEndDate'], how='left')
df_fn = df_fn.merge(df_ma, on=['SecuritiesCode', 'CurrentPeriodEndDate'], how='left')


In [217]:
df_fn[['TotalAssets', 'Equity']] = df_fn[['TotalAssets', 'Equity']].replace('－', 0).astype('float32')

In [218]:
amount_cols = ['TotalAssets', 'Equity']

amount_cols += ["diff_" + col for col in acc_cols]
amount_cols += ["ma_" + col for col in acc_cols]
amount_cols += acc_cols

key_cols = ["SecuritiesCode", "FiscalYear", "TypeOfCurrentPeriod"]
df_last_year = df_fn.groupby(key_cols)[amount_cols].nth(-1).reset_index()
df_last_year.FiscalYear += 1
df_last_fn = df_fn[key_cols].merge(df_last_year, on=key_cols, how='left')
df_last_ratio = (df_fn[amount_cols] - df_last_fn[amount_cols]) / df_last_fn[amount_cols]
df_last_ratio = df_last_ratio.add_prefix('feat_ratio_')


In [223]:
df_fn = pd.concat([df_fn, df_last_ratio], axis=1)

In [232]:
feature_stocklist_cols = ['Section/Products', '33SectorCode', 
    '17SectorCode', 'NewIndexSeriesSizeCode']

In [233]:
df_price_fe.merge(df_stocklist, on='SecuritiesCode', how='left')

,RowId,Date,SecuritiesCode,Open,High,Low,Close_x,Volume,AdjustmentFactor,ExpectedDividend,...,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,TradeDate,Close_y,IssuedShares,MarketCapitalization,Universe0
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,...,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
1,20170105_1301,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,...,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
2,20170106_1301,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,NaN,...,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
3,20170110_1301,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,NaN,...,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
4,20170111_1301,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,NaN,...,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,20190327_9997,2019-03-27,9997,892.0,897.0,883.0,887.0,180000,1.0,NaN,...,Retail Trade,14,RETAIL TRADE,6,TOPIX Small 1,20211230.0,709.0,97244472.0,6.894633e+10,True
1048571,20190328_9997,2019-03-28,9997,876.0,877.0,850.0,853.0,265500,1.0,NaN,...,Retail Trade,14,RETAIL TRADE,6,TOPIX Small 1,20211230.0,709.0,97244472.0,6.894633e+10,True
1048572,20190329_9997,2019-03-29,9997,857.0,872.0,853.0,858.0,289400,1.0,NaN,...,Retail Trade,14,RETAIL TRADE,6,TOPIX Small 1,20211230.0,709.0,97244472.0,6.894633e+10,True
1048573,20190401_9997,2019-04-01,9997,867.0,883.0,855.0,863.0,408800,1.0,NaN,...,Retail Trade,14,RETAIL TRADE,6,TOPIX Small 1,20211230.0,709.0,97244472.0,6.894633e+10,True


In [7]:
df_financial[df_financial.DisclosedDate != df_financial.Date]

,DisclosureNumber,DateCode,Date,SecuritiesCode,DisclosedDate,DisclosedTime,DisclosedUnixTime,TypeOfDocument,CurrentPeriodEndDate,TypeOfCurrentPeriod,...,ForecastEarningsPerShare,ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements,MaterialChangesInSubsidiaries,ChangesBasedOnRevisionsOfAccountingStandard,ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard,ChangesInAccountingEstimates,RetrospectiveRestatement,NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock,NumberOfTreasuryStockAtTheEndOfFiscalYear,AverageNumberOfShares
433,2.017012e+13,20170120_6634,2017-01-20,6634.0,2017-01-21,10:00:00,1.484960e+09,NumericalCorrection,2016-11-30,FY,...,-14.04,NaN,True,True,False,False,False,15030195.0,156058,14885214.0
4450,2.017033e+13,20170324_2146,2017-03-24,2146.0,2017-03-25,13:00:00,1.490414e+09,ForecastRevision,2017-03-31,FY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5055,2.017041e+13,20170414_9919,2017-04-14,9919.0,2017-04-15,15:00:00,1.492236e+09,ForecastRevision,2017-03-31,FY,...,27.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12573,2.017081e+13,20170810_3058,2017-08-10,3058.0,2017-08-11,15:10:00,1.502432e+09,1QFinancialStatements_Consolidated_JP,2017-06-30,1Q,...,8.5,NaN,False,False,False,False,False,6000000.0,117606,5882394.0
12930,2.017080e+13,20170810_7524,2017-08-10,7524.0,2017-08-11,15:00:00,1.502431e+09,1QFinancialStatements_NonConsolidated_JP,2017-06-30,1Q,...,7.48,False,NaN,False,False,False,False,8550400.0,521678,8028752.0
14307,2.017102e+13,20171020_9919,2017-10-20,9919.0,2017-10-21,15:00:00,1.508566e+09,ForecastRevision,2018-03-31,FY,...,15.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21576,2.018021e+13,20180209_7898,2018-02-09,7898.0,2018-02-12,09:00:00,1.518394e+09,ForecastRevision,2018-03-31,FY,...,10.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22557,NaN,NaN,2018-02-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23500,2.018041e+13,20180413_3185,2018-04-13,3185.0,2018-04-14,02:50:00,1.523642e+09,ForecastRevision,2018-03-31,FY,...,14.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26086,2.018051e+13,20180511_6264,2018-05-11,6264.0,2018-05-12,15:00:00,1.526105e+09,ForecastRevision,2018-08-31,FY,...,73.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
